In [1]:
# Add table of contents

# Libraries and Notebook Options

In [2]:
import pandas as pd
import numpy as np

# Exploratory Data Analysis
from pandas_profiling import ProfileReport

# # Visualizations
# from matplotlib.pyplot import figure
# from sklearn.metrics import roc_auc_score, roc_curve
# import matplotlib.pyplot as plt
# import seaborn as sns
# from mlxtend.plotting import plot_decision_regions
# %matplotlib inline

# SODA API

In [3]:
# from sodapy import Socrata
# # Unauthenticated client only works with public data sets. Note 'None'
# # in place of application token, and no username or password:
# client = Socrata('data.sfgov.org', None)

# # Authenticate the client
# domain = 'data.sfgov.org'
# app_token = 'tZuHs7ko1xQezRBAgUN10D0i9'
# username = 'corraljrmiguel@gmail.com'
# password = 'Xd9@rzwPhZjKsNg8cima'

# client = Socrata(domain,
#                  app_token,
#                  username=username,
#                  password=password)

# # Returns as JSON from API
# # converted to Python list of dictionaries by sodapy.
# results = client.get('vw6y-z8j6', limit=10000)
# # results = client.get('vw6y-z8j6', where='service_request_id=12167455')

# # Convert to pandas dataframe
# df = pd.DataFrame.from_records(results)

# Data Cleaning

In [4]:
# # Import CSV – 4.1M rows
# df = pd.read_csv('../data/311_Cases.csv')

# [TEMPORARY]
df = read_pickle('../data/df_raw_all.pkl')
df = df.sample(n=20000, random_state=2020)

# Sort dataframe by CaseID (newest first)
df.sort_values(by='CaseID', ascending=False, inplace=True)

# # Set index to CaseID
# df.set_index('CaseID', drop=True)

# Reset the index
df.reset_index(drop=True, inplace=True)

df.shape

(20000, 47)

In [5]:
# Drop all computed columns (unnecessary)
df = df.iloc[:, 0:19]
# df = df.loc[:, ~df.columns.str.contains('^:@')]

print(df.shape)
df.head()

(20000, 19)


,CaseID,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,Address,Street,Supervisor District,Neighborhood,Police District,Latitude,Longitude,Point,Source
0,12169918,03/03/2020 06:15:55 PM,NaN,03/03/2020 06:15:58 PM,Open,open,HSOC Queue,Encampments,Encampment Reports,Encampment Cleanup,"416 PORTOLA DR, SAN FRANCISCO, CA, 94131",PORTOLA DR,8.0,Midtown Terrace,INGLESIDE,37.746743,-122.448544,"(37.74674321, -122.44854368)",Mobile/Open311
1,12169593,03/03/2020 05:07:06 PM,NaN,03/03/2020 05:32:01 PM,Open,accepted,DPT SignShop Queue,Sign Repair,Sign - Defaced,Other_Parking_Type - Defaced; support - OK,"3450 JUDAH ST, SAN FRANCISCO, CA, 94122",JUDAH ST,4.0,Outer Sunset,TARAVAL,37.760813,-122.499170,"(37.76081333, -122.49917)",Mobile/Open311
2,12168442,03/03/2020 01:57:00 PM,03/03/2020 02:10:54 PM,03/03/2020 02:10:54 PM,Closed,Case Resolved - Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Boxed or Bagged Items,Intersection of DWIGHT ST and GIRARD ST,DWIGHT ST,9.0,Portola,BAYVIEW,37.723774,-122.403107,"(37.72377396, -122.40310669)",Phone
3,12168325,03/03/2020 01:36:00 PM,NaN,03/03/2020 01:45:07 PM,Open,open,MUNI Work Queue,MUNI Feedback,MUNI - Conduct_Inattentiveness_Negligence,201_Pass_Up_Did_Not_Wait_for_Transferee,Intersection of MISSION ST and 30TH ST,MISSION ST,9.0,Bernal Heights,INGLESIDE,37.742374,-122.421967,"(37.74237442, -122.42196655)",Phone
4,12167749,03/03/2020 12:01:00 PM,NaN,03/03/2020 12:25:21 PM,Open,accepted,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,"3373 MISSION ST, SAN FRANCISCO, CA, 94110",MISSION ST,9.0,Bernal Heights,INGLESIDE,37.742619,-122.421705,"(37.74261877, -122.42170504)",Mobile/Open311


In [6]:
# Strip column names and format
df.columns = df.columns.str.strip().str.lower().str.replace(
    ' ', '_').str.replace('(', '').str.replace(')', '')

# Rename 'caseid' column
df.rename({'caseid': 'case_id'}, axis='columns', inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 19 columns):
case_id                20000 non-null int64
opened                 20000 non-null object
closed                 19561 non-null object
updated                20000 non-null object
status                 20000 non-null object
status_notes           19855 non-null object
responsible_agency     20000 non-null object
category               20000 non-null object
request_type           20000 non-null object
request_details        19672 non-null object
address                20000 non-null object
street                 18310 non-null object
supervisor_district    18310 non-null float64
neighborhood           18295 non-null object
police_district        18285 non-null object
latitude               20000 non-null float64
longitude              20000 non-null float64
point                  20000 non-null object
source                 20000 non-null object
dtypes: float64(3), int64(1), obje

In [7]:
# Convert time columns to datetime
df['opened'] = pd.to_datetime(df['opened'])
df['closed'] = pd.to_datetime(df['closed'])
df['updated'] = pd.to_datetime(df['updated'])
# df.iloc[:, 1:4] = pd.to_datetime(df.iloc[:, 1:4].stack()).unstack()

df.shape

(20000, 19)

In [8]:
# Remove incomplete years (2008 & 2020)
df = df.loc[(df['opened'] >= '2009-01-01') & (df['opened'] <= '2019-12-31')]

# Drop all rows with any missing value
df.dropna(how='any', inplace=True)

# Drop 'status' column as all incidents are 'closed' after running the code above
df.drop('status', axis=1, inplace=True)

df.shape

(16786, 18)

In [9]:
# Convert categorical columns to 'category' type
df['supervisor_district'] = pd.to_numeric(df['supervisor_district'], downcast='integer').astype('category')

cols = df[['category', 'neighborhood', 'police_district', 'source']]
for col in cols:
    df[col] = df[col].astype('category')
    
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16786 entries, 619 to 19685
Data columns (total 18 columns):
case_id                16786 non-null int64
opened                 16786 non-null datetime64[ns]
closed                 16786 non-null datetime64[ns]
updated                16786 non-null datetime64[ns]
status_notes           16786 non-null object
responsible_agency     16786 non-null object
category               16786 non-null category
request_type           16786 non-null object
request_details        16786 non-null object
address                16786 non-null object
street                 16786 non-null object
supervisor_district    16786 non-null category
neighborhood           16786 non-null category
police_district        16786 non-null category
latitude               16786 non-null float64
longitude              16786 non-null float64
point                  16786 non-null object
source                 16786 non-null category
dtypes: category(5), datetime64[ns](3), floa

In [10]:
# Create 'is_duplicate' column
def create_is_duplicate(df):
    """Create 'is_duplicate' column"""
    
    # Search for duplicates in 'status_notes'
    df.insert(loc=1, column='is_duplicate', 
              value=df['status_notes'].str.lower().str.contains('dupe|duplicate', regex=True).astype('int'))
    
    # Fill 'nan' values with False
    df['is_duplicate'].fillna(False, inplace=True)
    
    return df

create_is_duplicate(df)

# Move target variable to 2nd index


df.head()

,case_id,is_duplicate,opened,closed,updated,status_notes,responsible_agency,category,request_type,request_details,address,street,supervisor_district,neighborhood,police_district,latitude,longitude,point,source
619,11879423,0,2019-12-30 20:54:00,2020-01-03 11:59:19,2020-01-03 11:59:19,Agencies responded to request and no encampmen...,Duplicate Case Hold Queue,Encampments,Encampment Reports,Encampment Cleanup,"3983 19TH ST, SAN FRANCISCO, CA, 94114",19TH ST,8,Dolores Heights,MISSION,37.759285,-122.432162,"(37.759285, -122.432162)",Mobile/Open311
620,11877576,0,2019-12-30 13:31:54,2019-12-31 13:42:03,2019-12-31 13:42:03,Case Resolved - SES Graffiti Crew - Remove Si...,DPW Ops Queue,Illegal Postings,Illegal Postings - Affixed_Improperly,Affixed Improperly,"111 PAGE ST, SAN FRANCISCO, CA, 94102",PAGE ST,5,Lower Haight,NORTHERN,37.773851,-122.423080,"(37.77385123, -122.42307977)",Mobile/Open311
621,11877532,0,2019-12-30 13:26:00,2019-12-30 14:03:00,2019-12-30 14:03:00,Case Resolved,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,Intersection of OSAGE ALY and 26TH ST,OSAGE ALY,9,Mission,MISSION,37.749008,-122.418709,"(37.74900818, -122.4187088)",Phone
622,11877496,0,2019-12-30 13:22:00,2019-12-30 18:53:45,2019-12-30 18:53:45,Case Resolved - WASTE NOT FOUND ...,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Refrigerator,Intersection of OSAGE ALY and 25TH ST,OSAGE ALY,9,Mission,MISSION,37.750607,-122.418861,"(37.75060654, -122.41886139)",Phone
623,11877234,0,2019-12-30 12:45:38,2020-01-02 07:17:00,2020-01-02 07:17:00,Case Resolved,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,Intersection of ELGIN PARK and MCCOPPIN ST,ELGIN PARK,8,Mint Hill,SOUTHERN,37.771481,-122.423325,"(37.77148065, -122.4233246)",Mobile/Open311


In [11]:
# profile = ProfileReport(df, title='Pandas Profiling Report', html={'style':{'full_width':True}})
# profile.to_notebook_iframe()

In [12]:
# Pickle dataframe
df.to_pickle('../data/df_clean_20k.pkl')